In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Posts(2).csv")

In [3]:
df['Text'] = df['Text'].str.lower()

df['Text'] = df['Text'].str.replace('(#(\w)+)', '')
df['Text'] = df['Text'].str.replace('(@(\w)+)', '')

df['Text'] = df['Text'].str.replace('&', 'and')
df['Text'] = df['Text'].str.replace('a-z0-9\.\?\:\. |', '')
df['Text'] = df['Text'].str.replace('<[^<>]+>', '')
df['Text'] = df['Text'].str.replace('[0-9]+', 'number')
df['Text'] = df['Text'].str.replace('(http|https)://[^\s]*', '')
df['Text'] = df['Text'].str.replace('[\.]+', '.')
df['Text'] = df['Text'].str.replace('[\,]+', ',')
df['Text'] = df['Text'].str.replace('[\?]+', '?')
df['Text'] = df['Text'].str.replace('[\!]+', '!')

#df['Text'] = df['Text'].str.replace('yall', 'you all')
#df['Text'] = df['Text'].str.replace('im', 'i am')
#df['Text'] = df['Text'].str.replace('id', 'i would')
#df['Text'] = df['Text'].str.replace('hes', 'he is')
#df['Text'] = df['Text'].str.replace('its', 'it is')
#df['Text'] = df['Text'].str.replace('isnt', 'is not')
#df['Text'] = df['Text'].str.replace('arent', 'are not')
#df['Text'] = df['Text'].str.replace('lemme', 'let me')
#df['Text'] = df['Text'].str.replace('imma', 'i am going to')
#df['Text'] = df['Text'].str.replace('cant', 'can not')
#df['Text'] = df['Text'].str.replace('dont', 'do not')
#df['Text'] = df['Text'].str.replace('didnt', 'did not')
#df['Text'] = df['Text'].str.replace('doesnt', 'does not')
#df['Text'] = df['Text'].str.replace('wouldnt', 'would not')
#df['Text'] = df['Text'].str.replace('hasnt', 'has not')
#df['Text'] = df['Text'].str.replace('havent', 'have not')
#df['Text'] = df['Text'].str.replace('gonna', 'going to')
#df['Text'] = df['Text'].str.replace('gotta', 'got to')
#df['Text'] = df['Text'].str.replace('idk', 'i do not know')
#df['Text'] = df['Text'].str.replace('kms', 'kill myself')
#df['Text'] = df['Text'].str.replace('tf', 'the fuck')
#df['Text'] = df['Text'].str.replace('wtf', 'what the fuck')
#df['Text'] = df['Text'].str.replace('stfu', 'shut the fuck up')
#df['Text'] = df['Text'].str.replace('fk', 'fuck')
#df['Text'] = df['Text'].str.replace('rly', 'really')

In [4]:
#from nltk.corpus import stopwords
#nltk.download('stopwords')
#from nltk.tokenize import word_tokenize

#for text in df:
#    text_tokens = word_tokenize(text)
#    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]

In [80]:
x_data = df['Text'].values
y_data = df['Ideation'].values
x_data

array(['a life without coffee is a life not worth living.wisdom of quarantine',
       'life without risks is not worth living.',
       'living your life in fear is a life not worth living.', ...,
       'i was born crying but i wanna die smiling',
       'i was born alone but i wanna die with the one i love the most',
       'god forgive me, but i wanna die!'], dtype=object)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.25, random_state = 42)

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [79]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(x_train)

X_train = tokenizer.texts_to_sequences(x_train)
X_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen = 50
X_train = pad_sequences(X_train, padding = 'post', maxlen = maxlen)
X_test = pad_sequences(X_test, padding = 'post', maxlen = maxlen)

In [11]:
encoder = OneHotEncoder(sparse = False)

y_train = y_train.reshape((len(y_train), 1))
y_train = encoder.fit_transform(y_train)

y_test = y_test.reshape((len(y_test), 1))
y_test = encoder.fit_transform(y_test)

In [12]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)
    
    plt.figure(figsize = (12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label = 'Training accuracy')
    plt.plot(x, val_acc, 'r', label = 'Validation accuracy')
    plt.title('Training and validation acccuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label = 'Training loss')
    plt.plot(x, val_loss, 'r', label = 'Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [13]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    
    with open(filepath, encoding = 'utf8') as f:
        for line in f:
            values = line.split()
            if values[0] in word_index:
                idx = word_index[values[0]]
                embedding_matrix[idx] = np.asarray(values[1:embedding_dim+1], 'float32')
                
    return embedding_matrix

In [14]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('glove.twitter.27B.50d.txt', 
                                           tokenizer.word_index, 
                                           embedding_dim)

In [15]:
#LSTM

model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, 
                           output_dim = embedding_dim,
                           weights = [embedding_matrix],
                           input_length = maxlen,
                           trainable = True))
model.add(layers.Conv1D(32, 3, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(pool_size = 3, strides = 1, padding = 'valid'))
model.add(layers.Dropout(0.3))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(3, activation = 'softmax'))

In [33]:
#GRU

model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, 
                           output_dim = embedding_dim,
                           weights = [embedding_matrix],
                           input_length = maxlen,
                           trainable = True))
model.add(layers.Conv1D(48, 3, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(3, 1, padding = 'valid'))
model.add(layers.Dropout(0.3))
model.add(layers.GRU(48, return_sequences = True))
#model.add(layers.GRU(12, return_sequences = False))
#model.add(layers.Dropout(0.3))
model.add(layers.SimpleRNN(12))
model.add(layers.Dense(3, activation = 'softmax'))

In [20]:
#CNN

model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, 
                           output_dim = embedding_dim,
                           weights = [embedding_matrix],
                           input_length = maxlen,
                           trainable = True))
model.add(layers.Conv1D(32, 4, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(32, 3, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(32, 2, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(3, activation = 'softmax'))

In [21]:
#Mixed

model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, 
                           output_dim = embedding_dim,
                           weights = [embedding_matrix],
                           input_length = maxlen,
                           trainable = True))
model.add(layers.Conv1D(32, 4, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.3))
model.add(layers.Conv1D(32, 3, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.3))
model.add(layers.Conv1D(32, 2, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.3))
model.add(layers.GRU(32, return_sequences = True))
model.add(layers.GRU(16, return_sequences = False))
model.add(layers.Dense(3, activation = 'softmax'))

In [34]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = [tf.keras.metrics.RecallAtPrecision(0.80)])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 50)            102050    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 48, 48)            7248      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 46, 48)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 46, 48)            0         
_________________________________________________________________
gru_2 (GRU)                  (None, 46, 48)            14112     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 12)                732       
_________________________________________________________________
dense_4 (Dense)              (None, 3)                

In [49]:
history = model.fit(X_train, y_train,
                    epochs = 5,
                    verbose = 1,
                    validation_data = (X_test, y_test),
                    batch_size = 1)

Epoch 1/5
989/989 [==============================] - 24s 24ms/step - loss: 0.2183 - recall_at_precision_2: 0.9727 - val_loss: 1.4951 - val_recall_at_precision_2: 0.0000e+00
Epoch 2/5
989/989 [==============================] - 24s 24ms/step - loss: 0.1275 - recall_at_precision_2: 0.9929 - val_loss: 1.5848 - val_recall_at_precision_2: 0.0000e+00
Epoch 3/5
989/989 [==============================] - 24s 24ms/step - loss: 0.1596 - recall_at_precision_2: 0.9899 - val_loss: 1.4193 - val_recall_at_precision_2: 0.0000e+00
Epoch 4/5
989/989 [==============================] - 27s 27ms/step - loss: 0.1063 - recall_at_precision_2: 0.9919 - val_loss: 1.5159 - val_recall_at_precision_2: 0.0000e+00
Epoch 5/5
989/989 [==============================] - 25s 25ms/step - loss: 0.1132 - recall_at_precision_2: 0.9929 - val_loss: 1.3929 - val_recall_at_precision_2: 0.3273


In [50]:
prediction_train = [i.argmax() for i in model.predict(X_train)]
prediction_test = [i.argmax() for i in model.predict(X_test)]

Y_train_hat = np.array(prediction_train)
Y_test_hat = np.array(prediction_test)
Y_train = np.array([i.argmax() for i in y_train])
Y_test = np.array([i.argmax() for i in y_test])

tp = sum((Y_test == Y_test_hat) & (Y_test == 2))
tn = sum((Y_test == Y_test_hat) & (Y_test != 2))
fp = sum((Y_test != Y_test_hat) & (Y_test_hat == 2))
fn = sum((Y_test != Y_test_hat) & (Y_test == 2))

precision = tp/(tp+fp)
recall = tp/(tp+fn)

f1 = 2 * precision * recall / (precision + recall)
precision, recall, f1

(0.7611940298507462, 0.6710526315789473, 0.7132867132867132)

In [51]:
loss, accuracy = model.evaluate(X_train, y_train, verbose = False)
print('Training Accuracy: {:.4f}'.format(accuracy))

loss, accuracy = model.evaluate(X_test, y_test, verbose = False)
print('Testing Accuracy: {:.4f}'.format(accuracy))

Training Accuracy: 0.9980
Testing Accuracy: 0.3273


In [27]:
plot_history(history)

KeyError: 'accuracy'

In [ ]:
#Type, Training Accuracy, Testing Accuracy, precision, recall, F1 - twitter.27b.50d, embedding_dim = 50
#LSTM, 0.9986, 0.6992, 0.48, 0.5714, 0.5217
#GRU, 0.9918, 0.7033, 0.5417, 0.6190, 0.5778
#CNN, 0.9973, 0.6260, 0.4444, 0.5714, 0.5

#Type, Training Accuracy, Testing Accuracy, precision, recall, F1 - twitter.27b.200d, embedding_dim = 200
#LSTM, 1.0000, 0.6870, 0.5882, 0.4762, 0.5263
#GRU, 0.9986, 0.6667, 0.625, 0.4762, 0.5405
#CNN, 0.9973, 0.7195, 0.3939, 0.6190, 0.4815

#Type, Training Accuracy, Testing Accuracy, precision, recall, F1 - twitter.27b.50d, embedding_dim = 50
#GRU (Metrics = tf.keras.metrics.AUC()), N/A, N/A, 0.5652, 0.6190, 0.5909

#Type, Training Accuracy, Testing Accuracy, precision, recall F1 - twitter.27b.50d, embedding_dim = 50, new data
#GRU (Metric = tf.keras.metrics.AUC()), N/A, N/A, 0.8605, 0.5441, 0.6666

In [20]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model

In [21]:
# serialize model to JSON
#  the keras model which is trained is defined as 'model' in this example
model_json = model.to_json()


with open("model_.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("model_.h5")

In [22]:
json_file = open('model_800.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model_800.h5")

In [23]:
prediction_train = [i.argmax() for i in loaded_model.predict(X_train)]
prediction_test = [i.argmax() for i in loaded_model.predict(X_test)]

Y_train_hat = np.array(prediction_train)
Y_test_hat = np.array(prediction_test)
Y_train = np.array([i.argmax() for i in y_train])
Y_test = np.array([i.argmax() for i in y_test])

tp = sum((Y_test == Y_test_hat) & (Y_test == 2))
tn = sum((Y_test == Y_test_hat) & (Y_test != 2))
fp = sum((Y_test != Y_test_hat) & (Y_test_hat == 2))
fn = sum((Y_test != Y_test_hat) & (Y_test == 2))

precision = tp/(tp+fp)
recall = tp/(tp+fn)

f1 = 2 * precision * recall / (precision + recall)
precision, recall, f1

(0.8108108108108109, 0.7894736842105263, 0.8)

In [86]:
test = "I kill myself"
test = test.lower()

test = test.replace('(#(\w)+)', '')
test = test.replace('(@(\w)+)', '')

test = test.replace('&', 'and')
test = test.replace('a-z0-9\.\?\:\. |', '')
test = test.replace('<[^<>]+>', '')
test = test.replace('[0-9]+', 'number')
test = test.replace('(http|https)://[^\s]*', '')
test = test.replace('[\.]+', '.')
test = test.replace('[\,]+', ',')
test = test.replace('[\?]+', '?')
test = test.replace('[\!]+', '!')

test = tokenizer.texts_to_sequences(test)
test

[[1], [], [1262], [1], [1775], [1775], [], [], [], [], [], [1775], []]